In [1]:
from itertools import chain

import numpy as np
import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer,classification_report
from sklearn.model_selection import cross_val_score, RandomizedSearchCV, train_test_split, cross_val_predict

import sklearn_crfsuite
from sklearn_crfsuite import scorers, CRF, metrics

In [2]:
import pandas as pd

data = pd.read_csv('final_dataset.csv', encoding='latin1', low_memory=False, dtype={'Sentence #': str, 'Word': str, 'POS': str, 'Tag': str})
filled_data = data.fillna(method='ffill')

class SentenceGetter(object):
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                            s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
  
    def get_next(self):     
        try:
             s = self.grouped["Sentence: {}".format(self.n_sent)]
             self.n_sent += 1
             return s
        except:
             return None       

getter = SentenceGetter(filled_data)
sentences = getter.sentences

In [3]:
%%time
# train_sents = list(nltk.corpus.conll2002.iob_sents('esp.train'))
# test_sents = list(nltk.corpus.conll2002.iob_sents('esp.testb'))

def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

# X = [sent2features(s) for s in train_sents]
# y = [sent2labels(s) for s in train_sents]

# X_test = [sent2features(s) for s in test_sents]
# y_test = [sent2labels(s) for s in test_sents]

X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# print(X_test[7])

Wall time: 385 ms


In [4]:
%%time
crf = CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=False
)
crf.fit(X_train, y_train)

# pred = cross_val_predict(estimator=crf, X=X, y=y, cv=5, n_jobs=-1, verbose=1)
# report = classification_report(y_pred=pred, y_true=y)
# print(report)

labels = list(crf.classes_)
labels.remove('O')

Wall time: 13.5 s


In [5]:
print(labels)

['B-MAL', 'B-OS', 'B-DVEC', 'I-DVEC', 'B-BEH', 'B-CAP', 'I-CAP', 'I-BEH', 'I-MAL', 'I-OS']


In [6]:
%%time
# define fixed parameters and parameters to search
crf = CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=False
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.5),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=labels)

# search
rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=4,
                        n_jobs=-1,
                        n_iter=20,
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  8.9min finished


Wall time: 9min 8s


In [7]:
import pickle
filename = '1_crf_opti_model.sav'
pickle.dump(rs, open(filename, 'wb'))

In [8]:
print('best params:', rs.best_params_)
print('best cv score:', rs.best_score_)
print('model size {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

best params: {'c1': 0.041906528896993624, 'c2': 0.01433007013268679}
best cv score: 0.8410489533317788
model size 0.25M


In [9]:
crf = rs.best_estimator_
y_pred = crf.predict(X_test)

sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)

print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

              precision    recall  f1-score   support

       B-BEH      0.922     0.808     0.861       146
       I-BEH      1.000     0.750     0.857        32
       B-CAP      0.982     0.900     0.939        60
       I-CAP      0.979     0.939     0.958        49
      B-DVEC      0.875     0.778     0.824        63
      I-DVEC      0.932     0.788     0.854        52
       B-MAL      0.909     0.964     0.936       249
       I-MAL      1.000     1.000     1.000         1
        B-OS      0.861     0.939     0.899        33
        I-OS      0.818     1.000     0.900         9

   micro avg      0.920     0.883     0.901       694
   macro avg      0.928     0.887     0.903       694
weighted avg      0.922     0.883     0.900       694



In [10]:
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f"%(label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])


Top likely transitions:
B-DVEC -> I-DVEC  5.639431
I-BEH  -> I-BEH   5.086817
B-CAP  -> I-CAP   4.950763
B-OS   -> I-OS    4.451265
B-MAL  -> I-MAL   3.757583
B-BEH  -> I-BEH   3.743590
O      -> O       3.610088
I-OS   -> I-OS    3.521754
I-CAP  -> I-CAP   3.433305
I-DVEC -> I-DVEC  3.014713
I-MAL  -> I-MAL   2.414155
O      -> B-BEH   1.427862
O      -> B-MAL   1.196830
O      -> B-DVEC  1.113900
B-MAL  -> B-BEH   1.051131
O      -> B-CAP   0.852822
B-MAL  -> O       0.750119
B-CAP  -> B-MAL   0.359316
B-BEH  -> O       0.217070
I-CAP  -> O       0.187649

Top unlikely transitions:
B-BEH  -> O       0.217070
I-CAP  -> O       0.187649
O      -> B-OS    0.093559
B-CAP  -> B-CAP   0.027732
B-CAP  -> B-DVEC  -0.068609
B-OS   -> O       -0.279608
I-DVEC -> O       -0.340921
I-MAL  -> O       -0.684048
I-BEH  -> O       -0.747532
B-CAP  -> O       -0.815387
B-DVEC -> B-DVEC  -0.957932
B-MAL  -> B-OS    -1.123167
B-OS   -> B-DVEC  -1.225512
B-DVEC -> I-BEH   -1.505942
I-OS   -> O       -1.

In [11]:
import eli5

eli5.show_weights(crf, top=30)

From \ To,O,B-BEH,I-BEH,B-CAP,I-CAP,B-DVEC,I-DVEC,B-MAL,I-MAL,B-OS,I-OS
O,3.61,1.428,-6.163,0.853,-3.201,1.114,-2.498,1.197,0.0,0.094,0.0
B-BEH,0.217,0.0,3.744,0.0,0.0,-3.582,0.0,0.0,0.0,0.0,0.0
I-BEH,-0.748,0.0,5.087,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-CAP,-0.815,0.0,0.0,0.028,4.951,-0.069,0.0,0.359,0.0,0.0,0.0
I-CAP,0.188,0.0,0.0,0.0,3.433,0.0,0.0,0.0,0.0,0.0,0.0
B-DVEC,-2.595,0.0,-1.506,0.0,0.0,-0.958,5.639,0.0,0.0,0.0,0.0
I-DVEC,-0.341,0.0,0.0,0.0,0.0,0.0,3.015,0.0,0.0,0.0,0.0
B-MAL,0.75,1.051,0.0,0.0,0.0,0.0,0.0,0.0,3.758,-1.123,0.0
I-MAL,-0.684,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.414,0.0,0.0
B-OS,-0.28,0.0,0.0,0.0,0.0,-1.226,0.0,0.0,0.0,0.0,4.451
